In [1]:
## This script is to calculate sensitivity of the site at initial
## to the domain at future forecast hour (fh) ;; input from CALCUL_PWVgrid_V02.ipynb
## ;;; move manually to stratus
## Created by C. Bayu Risanto, S.J. (23 February 2024)
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import random
import pandas as pd
import xarray as xr
from scipy.interpolate import griddata
from scipy.io import savemat
from scipy import stats
from sklearn.metrics import mean_squared_error
from math import sqrt
import math as math
import time
import glob
from datetime import date, datetime, timedelta
## opener
def read_nx(ncfile):
    dx = xr.open_dataset(ncfile)
    pwv = dx.PWV[:,:]
    return pwv

In [2]:
## read grid data of forecast hours
path = '/net/stratus/st8/bayu/ARIZONA-WRF/DART_runs/'
dirin = '20210815_06Z_40mem_06Z_0035_6cyc_noassim'
to_fh = '2021-08-15_18:00'
fis = glob.glob(path+dirin+'/'+'extract'+'/'+'mem*'+'/'+'PWV_d02_'+to_fh+'.nc') 

## collect data
pwv = [read_nx(fis[iens]) for iens in range(len(fis))]
PWV = np.asarray(pwv)  ; print(PWV.shape)

(40, 405, 684)


In [3]:
## read site data of initial forecast
## loop carefully. 
FCT_all = []
for iens in range(40):
    dfileCY = 'GPSequiv_CYC_staENS'+str(iens+1).zfill(2)+'_pwv.dat'
    with open(path+dirin+'/'+'extract'+'/'+dfileCY,'r') as file:
        # Read the contents of the original file
        file_contentsCY = file.readlines()
    
    # Process and modify the data
    val_dataCY = []; sitesCY = []; latlonCY = []; timeCY = []
    for line in file_contentsCY:
        parts = line.strip().split(",")  # Split the line using commas as the delimiter
        if len(parts) == 6:
            stname = parts[0]
            hgt = float(parts[1])
            timestamp = parts[4]
            value = float(parts[5])
            val_dataCY.append(value)
            sitesCY.append(stname)
            timeCY.append(timestamp)
    # make array of sites X times AND array of nens X sites X times
    Fct_pwv = []
    SITE = sitesCY[0:33]
    st = np.asarray(sitesCY)
    for isite in range(len(SITE)):
        ixst = np.asarray(np.where(st == SITE[isite]))
        allpwvCY = np.asarray(val_dataCY)
        st_pwvCY = np.squeeze(allpwvCY[ixst])
        Fct_pwv.append(st_pwvCY)
    Fct_pwv = np.asarray(Fct_pwv)
    FCT_all.append(Fct_pwv)  
FCT_all = np.asarray(FCT_all)  ; #print(FCT_all.shape)

In [4]:
## Let's calculate sensitivity of the domain at certain time to the change of the site at initial condition
itim = 0  #suggest: keep it initial time (i.e.,0)
ista = 31  #change the site here!!!!!!!!!!

##*********** BE SUPER CAREFUL in MODIFYING *********************************************
nens,ny,nx = PWV.shape
sentiv = [[(np.cov(PWV[:,jg,ig],FCT_all[:,ista,itim])[0,1]/np.var(FCT_all[:,ista,itim]))
           *(np.mean(FCT_all[:,ista,itim])/np.mean(PWV[:,jg,ig])) for jg in range(ny)] for ig in range(nx)]
SENSI = np.asarray(sentiv).T

## let's calculate p-value
value = float('nan')
pval = [[stats.pearsonr(PWV[:,jg,ig],FCT_all[:,ista,itim]) if SENSI[jg,ig] != value else value
         for jg in range(ny)] for ig in range(nx)] 
PVAL = np.asarray(pval)[:,:,1].T

##**************** END *******************************************************************

## save in mat file for easy storage
dir_out = '/home/bayu/Documents/ARIZONA-WRF/wrf_out/'+dirin+'/SENSITIVITY/'
output_file = dir_out + 'sensitive_PWV-'+to_fh+'-'+SITE[ista]+'.mat'
mdic = {'SENS':SENSI,'PVAL':PVAL}
savemat(output_file,mdic)

In [6]:
## make LOOP
itim = 0  #suggest: keep it initial time (i.e.,0)
A = np.arange(0,8,1)
B = np.arange(9,30,1)

for ins in B:
    ista = ins  #change the site here!!!!!!!!!! avoid 8, 30, and 32

##*********** BE SUPER CAREFUL in MODIFYING *********************************************
    nens,ny,nx = PWV.shape
    sentiv = [[(np.cov(PWV[:,jg,ig],FCT_all[:,ista,itim])[0,1]/np.var(FCT_all[:,ista,itim]))
               *(np.mean(FCT_all[:,ista,itim])/np.mean(PWV[:,jg,ig])) for jg in range(ny)] for ig in range(nx)]
    SENSI = np.asarray(sentiv).T

## let's calculate p-value
    value = float('nan')
    pval = [[stats.pearsonr(PWV[:,jg,ig],FCT_all[:,ista,itim]) if SENSI[jg,ig] != value else value
             for jg in range(ny)] for ig in range(nx)] 
    PVAL = np.asarray(pval)[:,:,1].T

##**************** END *******************************************************************

## save in mat file for easy storage
    dir_out = '/home/bayu/Documents/ARIZONA-WRF/wrf_out/'+dirin+'/SENSITIVITY/'
    output_file = dir_out + 'sensitive_PWV-'+to_fh+'-'+SITE[ista]+'.mat'
    mdic = {'SENS':SENSI,'PVAL':PVAL}
    savemat(output_file,mdic)
    
    print('save site '+str(ins))

save site 9
save site 10
save site 11
save site 12
save site 13
save site 14
save site 15
save site 16
save site 17
save site 18
save site 19
save site 20
save site 21
save site 22
save site 23
save site 24
save site 25
save site 26
save site 27
save site 28
save site 29


In [50]:
print(np.arange(9,30,1))

[ 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29]
